Generating landslides geojson from database of Conred containg all type of disaster in an excel

In [35]:
# Import libraries needed
import pandas as pd
from geojson import Feature, Point, FeatureCollection,dumps


In [36]:
PATH_INPUT_EXCEL = 'historico_derrumbes.xlsx'
EXCEL_PAGE_NAME = 'Filtro'
PATH_OUTPUT_GEOJSON = "data_landslides.geojson"

In [37]:
# We read Excel with Historic data of landslides and other events
historico = pd.read_excel(PATH_INPUT_EXCEL,EXCEL_PAGE_NAME)


In [38]:
# We filter for the events we need that are catalogued as landslides
historico_derrumbes = historico[historico.Tipo.isin(['Derrumbe','Deslizamiento','Flujo De Lodo'])]

In [26]:
historico_derrumbes.count()


Latitud         4230
Longitud        4230
Fecha           4230
Evento          4225
Tipo            4230
Departamento    4230
Municipio       4228
Incidente       4230
dtype: int64

In [27]:
# Check how many landslides we have for each year
landslides_total  = 0
for year in range(2008,2023):
    landslides_year_count = historico_derrumbes[historico_derrumbes.Fecha.dt.year==year]['Latitud'].count()
    print("Year {} has {} landslides.".format(year, landslides_year_count))
    landslides_total += landslides_year_count
print("Total of landslides {}".format(landslides_total))


Year 2008 has 231 landslides.
Year 2009 has 25 landslides.
Year 2010 has 928 landslides.
Year 2011 has 372 landslides.
Year 2012 has 103 landslides.
Year 2013 has 156 landslides.
Year 2014 has 297 landslides.
Year 2015 has 115 landslides.
Year 2016 has 71 landslides.
Year 2017 has 207 landslides.
Year 2018 has 125 landslides.
Year 2019 has 192 landslides.
Year 2020 has 1087 landslides.
Year 2021 has 279 landslides.
Year 2022 has 42 landslides.
Total of landslides 4230


In [41]:
# Check how many landslides we have for each department
landslides_total  = 0
for departamento in historico_derrumbes.Departamento.unique():
    landslides_department_count = historico_derrumbes[historico_derrumbes.Departamento==departamento]['Latitud'].count()
    print("Departmento {} has {} landslides.".format(departamento, landslides_department_count))
    landslides_total += landslides_department_count
print("Total of landslides {}".format(landslides_total))

Departmento Guatemala has 629 landslides.
Departmento Chimaltenango has 172 landslides.
Departmento Solola has 373 landslides.
Departmento Quetzaltenango has 171 landslides.
Departmento Totonicapan has 125 landslides.
Departmento Retalhuleu has 11 landslides.
Departmento Sacatepequez has 105 landslides.
Departmento Quiche has 440 landslides.
Departmento San Marcos has 227 landslides.
Departmento Jutiapa has 59 landslides.
Departmento Alta Verapaz has 475 landslides.
Departmento Jalapa has 65 landslides.
Departmento Santa Rosa has 65 landslides.
Departmento Chiquimula has 94 landslides.
Departmento Huehuetenango has 521 landslides.
Departmento Zacapa has 251 landslides.
Departmento Baja Verapaz has 97 landslides.
Departmento Izabal has 93 landslides.
Departmento El Progreso has 154 landslides.
Departmento Suchitepequez has 57 landslides.
Departmento Peten has 21 landslides.
Departmento Escuintla has 25 landslides.
Total of landslides 4230


In [28]:
# We now create a geojson from the data of landslides
features = []
for index,row in historico_derrumbes.iterrows():
    try:
        latitude = float(row['Latitud'])
        longitude = float(row['Longitud'])
        # Checking if points is inside our country "Guatemala" and data is correct
        row_properties = {
            'landslide_id':str(index), # index will be our landslide_id
            'latitude' : str(row['Latitud']),
            'longitude' : str(row['Longitud']),
            'event_date' : str(row['Fecha']).split(" ")[0],
            'landslide' : str(1),
        }
        row_point = Point(coordinates=(longitude,latitude))
        row_feature = Feature(geometry=row_point,properties=row_properties)
        features.append(row_feature)    
    except:
        "Any error while parsing a data point"
        pass


In [30]:
# Check final length of data points without erros
len(features)

4227

In [31]:
# We convert each feature point to a feature collection to pass it to a geojson
feature_collection_landslides = FeatureCollection(features=features)

In [32]:
dump_landslides = dumps(feature_collection_landslides)

In [34]:
f = open(PATH_OUTPUT_GEOJSON, "w")
f.write(dump_landslides)
f.close()
